# Cloudfight Coding Contest AI 2022

In [ ]:
# !pip install pandas
# !pip install matplotlib
# !pip install scikit-learn
# !pip install xgboost
# !pip install lightgbm

In [2]:
# Matrix and plots
import pandas as pd
import matplotlib.pyplot as plt
# Preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
# Models
from xgboost import XGBRegressor
DATA = r"abalone.data"
SEED = 2022

## Load data

In [4]:
df = pd.read_csv(DATA, header=None)
X = df.iloc[:,0:-1] # Get first k-1 cols
print(X.head())
y = df.iloc[:,-1] # Get last col
print(y.head())

   0      1      2      3       4       5       6      7
0  M  0.455  0.365  0.095  0.5140  0.2245  0.1010  0.150
1  M  0.350  0.265  0.090  0.2255  0.0995  0.0485  0.070
2  F  0.530  0.420  0.135  0.6770  0.2565  0.1415  0.210
3  M  0.440  0.365  0.125  0.5160  0.2155  0.1140  0.155
4  I  0.330  0.255  0.080  0.2050  0.0895  0.0395  0.055
0    15
1     7
2     9
3    10
4     7
Name: 8, dtype: int64


## Preprocessing

In [5]:
enc = OneHotEncoder()
X = enc.fit_transform(X)
print(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=SEED)

  (0, 2)	1.0
  (0, 69)	1.0
  (0, 193)	1.0
  (0, 266)	1.0
  (0, 1014)	1.0
  (0, 3145)	1.0
  (0, 4442)	1.0
  (0, 5386)	1.0
  (1, 2)	1.0
  (1, 48)	1.0
  (1, 173)	1.0
  (1, 265)	1.0
  (1, 584)	1.0
  (1, 2906)	1.0
  (1, 4337)	1.0
  (1, 5236)	1.0
  (2, 0)	1.0
  (2, 84)	1.0
  (2, 204)	1.0
  (2, 274)	1.0
  (2, 1261)	1.0
  (2, 3208)	1.0
  (2, 4523)	1.0
  (2, 5497)	1.0
  (3, 2)	1.0
  :	:
  (4173, 5586)	1.0
  (4174, 2)	1.0
  (4174, 98)	1.0
  (4174, 215)	1.0
  (4174, 288)	1.0
  (4174, 2026)	1.0
  (4174, 3722)	1.0
  (4174, 4814)	1.0
  (4174, 5673)	1.0
  (4175, 0)	1.0
  (4175, 103)	1.0
  (4175, 217)	1.0
  (4175, 277)	1.0
  (4175, 1909)	1.0
  (4175, 3733)	1.0
  (4175, 4761)	1.0
  (4175, 5653)	1.0
  (4176, 2)	1.0
  (4176, 120)	1.0
  (4176, 231)	1.0
  (4176, 286)	1.0
  (4176, 2647)	1.0
  (4176, 4189)	1.0
  (4176, 4975)	1.0
  (4176, 5936)	1.0


## Model training

In [18]:
xgb_model = XGBRegressor(random_state=SEED)
xgb_model.fit(X_train, y_train)
xgb_pred = xgb_model.predict(X_test)

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:0: object

## Prediction